# 📊 Лабораторна робота №2  
## 🧑‍💻 Наука про дані: підготовчий етап  

### 🎯 Мета роботи  
Ознайомлення з основними етапами роботи з даними:  
✅ Від постановки задачі до написання пояснювальної записки.  
✅ Вивчення постановки задачі та природи даних, над якими виконуються аналітичні операції.  

### 📌 Основні поняття  
- **🗃 Сирі дані (raw data)** – початкові, необроблені дані, отримані з різних джерел.  
- **⚙️ Підготовка даних (data preparation)** – процес очищення, перетворення та структурування даних для подальшого аналізу.  

---

👨‍🎓 **ФБ-35**  
✍️ **Яковенко Анастасія**


### Спочатку імпортую необхідні бібліотеки для подальшого використання

In [92]:
from datetime import datetime
import requests
import urllib.request
import os
import pandas as pd
from IPython.display import display
import math

# 📥 Автоматизоване завантаження VHI-індексу  

Для кожної адміністративної одиниці України необхідно завантажити **тестові структуровані файли**, що містять значення **VHI-індексу**.  

✅ **Процедура має бути автоматизована** – параметром передається **індекс (номер) області**.  
✅ **Іменування файлів** – до імені файлу додається **дата та час завантаження**.  
✅ **Обробка повторних запусків** – передбачено:  
   - **Довантаження нових даних** без втрати попередніх.  
   - **Уникнення колізій** при повторному запуску скрипту.  


In [93]:
def download_vhi_data(region_id):
    url = (
        f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/"
        f"get_TS_admin.php?country=UKR&provinceID={region_id}&year1=1981&year2=2020&type=Mean"
    )
    folder = "Files"
    os.makedirs(folder, exist_ok=True)

    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = os.path.join(folder, f"vhi_{region_id}_{timestamp}.csv")

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  
        
        with open(filename, "w", encoding="utf-8") as file:
            file.write(response.text)
        
        print(f" [OK] Дані для регіону {region_id} збережено у файл {filename}")

    except requests.exceptions.RequestException as e:
        print(f" [ERROR] {region_id}: {e}")

if __name__ == "__main__":
    for region_id in range(1, 28):
        download_vhi_data(region_id)


 [OK] Дані для регіону 1 збережено у файл Files\vhi_1_2025-03-11_14-37-56.csv
 [OK] Дані для регіону 2 збережено у файл Files\vhi_2_2025-03-11_14-37-57.csv
 [OK] Дані для регіону 3 збережено у файл Files\vhi_3_2025-03-11_14-37-58.csv
 [OK] Дані для регіону 4 збережено у файл Files\vhi_4_2025-03-11_14-37-58.csv
 [OK] Дані для регіону 5 збережено у файл Files\vhi_5_2025-03-11_14-37-59.csv
 [OK] Дані для регіону 6 збережено у файл Files\vhi_6_2025-03-11_14-38-00.csv
 [OK] Дані для регіону 7 збережено у файл Files\vhi_7_2025-03-11_14-38-01.csv
 [OK] Дані для регіону 8 збережено у файл Files\vhi_8_2025-03-11_14-38-02.csv
 [OK] Дані для регіону 9 збережено у файл Files\vhi_9_2025-03-11_14-38-03.csv
 [OK] Дані для регіону 10 збережено у файл Files\vhi_10_2025-03-11_14-38-03.csv
 [OK] Дані для регіону 11 збережено у файл Files\vhi_11_2025-03-11_14-38-04.csv
 [OK] Дані для регіону 12 збережено у файл Files\vhi_12_2025-03-11_14-38-05.csv
 [OK] Дані для регіону 13 збережено у файл Files\vhi_13_20

# 📥 Завантаження та обробка VHI-даних у DataFrame  

## 🎯 Опис завдання  
Необхідно **зчитати завантажені текстові файли** з індексом **VHI** у **DataFrame** (бібліотека `pandas`).  

## 📌 Вимоги до реалізації  
✔ **Автоматичне зчитування всіх файлів** із вказаної директорії.  
✔ **Коректне форматування даних** при завантаженні у DataFrame.  
✔ **Імена стовпців повинні бути зрозумілими**, без спеціальних символів або пробілів.  
✔ **Окрема процедура**, яка приймає **шлях до директорії** як параметр.   


In [94]:


def reader(directory="Files"):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_files = os.listdir(directory)
    dataframe = pd.DataFrame()

    for file in all_files:
        path = os.path.join(directory, file)
        try:
            df = pd.read_csv(path, header=1, names=headers)

            df = df[df["VHI"] != -1]

            df["area"] = file.split("_")[1]

            df["area"] = pd.to_numeric(df["area"].str.strip(), errors="coerce")

            df["Year"] = df["Year"].astype(str).str.replace("<tt><pre>", "", regex=False)
            df = df[~df["Year"].str.contains('</pre></tt>', na=False)]

            df.drop('empty', axis=1, inplace=True, errors="ignore")

            dataframe = pd.concat([dataframe, df], ignore_index=True).drop_duplicates()

        except Exception as e:
            print(f"Error reading file {file}: {e}")

    dataframe["area"] = dataframe["area"].astype(int)
    dataframe = dataframe.sort_values(by=["area", "Year", "Week"]).reset_index(drop=True)

    return dataframe

df = reader("Files")




In [95]:

display(df)

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,1.0,0.053,260.31,45.01,39.46,42.23,1
1,1982,2.0,0.054,262.29,46.83,31.75,39.29,1
2,1982,3.0,0.055,263.82,48.13,27.24,37.68,1
3,1982,4.0,0.053,265.33,46.09,23.91,35.00,1
4,1982,5.0,0.050,265.66,41.46,26.65,34.06,1
...,...,...,...,...,...,...,...,...
53401,2020,48.0,0.115,269.40,57.48,31.51,44.49,27
53402,2020,49.0,0.102,267.62,52.70,33.89,43.29,27
53403,2020,50.0,0.095,266.71,50.56,30.20,40.38,27
53404,2020,51.0,0.084,265.81,44.95,26.76,35.86,27


# 🔄 Оновлення індексів областей NOAA на українські  

## 📌 Опис завдання  
Необхідно розробити процедуру, яка замінить індекси областей, використані на порталі **NOAA** (за англійською абеткою), на відповідні українські.  

## 🗂 Відповідність індексів  
| № NOAA | Область            | № Україна | Область         |
|--------|--------------------|-----------|-----------------|
| 1      | Вінницька         | 13        | Миколаївська   |
| 2      | Волинська         | 14        | Одеська        |
| 3      | Дніпропетровська  | 15        | Полтавська     |
| 4      | Донецька          | 16        | Рівненська     |
| 5      | Житомирська       | 17        | Сумська        |
| 6      | Закарпатська      | 18        | Тернопільська  |
| 7      | Запорізька        | 19        | Харківська     |
| 8      | Івано-Франківська | 20        | Херсонська     |
| 9      | Київська          | 21        | Хмельницька    |
| 10     | Кіровоградська    | 22        | Черкаська      |
| 11     | Луганська         | 23        | Чернівецька    |
| 12     | Львівська         | 24        | Чернігівська   |
| 25     | Республіка Крим   | 25        | Республіка Крим |

## 🎯 Вимоги до реалізації  
✔ **Автоматична заміна всіх індексів** у переданому DataFrame.  
✔ **Збереження інших даних без змін**.  
✔ **Перевірка на наявність потрібного стовпця (`Region_ID`)**.  
✔ **Безпечна обробка випадків, коли індексу немає у відповідності**.  

 


In [96]:
region_map = {
        24: "Вінницька", 25: "Волинська", 5: "Дніпропетровська", 6: "Донецька",
        27: "Житомирська", 23: "Закарпатська", 26: "Запорізька", 7: "Івано-Франківська",
        11: "Київська", 12: "Київ", 13: "Кіровоградська", 14: "Луганська", 15: "Львівська",
        16: "Миколаївська", 17: "Одеська", 18: "Полтавська", 19: "Рівненська", 20: "Севастополь",
        21: "Сумська", 22: "Тернопільська", 23: "Закарпаття", 8: "Харківська", 9: "Херсонська",
        10: "Хмельницька", 1: "Черкаська", 2: "Чернігівська", 3: "Чернівецька",
        4: "Республіка Крим"
    }

def map_region_names(df, id_column="area"):
    df[id_column] = df[id_column].astype(int)
    df["name"] = df[id_column].map(region_map)
    return df

df = map_region_names(df)
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area         name
0      1982   1.0  0.053  260.31  45.01  39.46  42.23     1    Черкаська
1      1982   2.0  0.054  262.29  46.83  31.75  39.29     1    Черкаська
2      1982   3.0  0.055  263.82  48.13  27.24  37.68     1    Черкаська
3      1982   4.0  0.053  265.33  46.09  23.91  35.00     1    Черкаська
4      1982   5.0  0.050  265.66  41.46  26.65  34.06     1    Черкаська
...     ...   ...    ...     ...    ...    ...    ...   ...          ...
53401  2020  48.0  0.115  269.40  57.48  31.51  44.49    27  Житомирська
53402  2020  49.0  0.102  267.62  52.70  33.89  43.29    27  Житомирська
53403  2020  50.0  0.095  266.71  50.56  30.20  40.38    27  Житомирська
53404  2020  51.0  0.084  265.81  44.95  26.76  35.86    27  Житомирська
53405  2020  52.0  0.077  264.70  42.45  27.64  35.05    27  Житомирська

[53406 rows x 9 columns]


In [97]:

ukr_region_order = [
    "Вінницька", "Волинська", "Дніпропетровська", "Донецька", "Житомирська",
    "Закарпаття", "Запорізька", "Івано-Франківська", "Київська", "Кіровоградська",
    "Луганська", "Київ", "Львівська", "Миколаївська", "Одеська", "Полтавська",
    "Рівненська",  "Севастополь", "Сумська", "Тернопільська", "Харківська", "Херсонська",
    "Хмельницька", "Черкаська", "Чернігівська", "Чернівецька", "Республіка Крим"
]

def sort_by_ukr_alphabet(df: pd.DataFrame, col_name: str = "region_name") -> pd.DataFrame:
   
    df[col_name] = pd.Categorical(df[col_name], categories=ukr_region_order, ordered=True)

    df = df.sort_values(by=col_name).reset_index(drop=True)
    return df

df = sort_by_ukr_alphabet(df, "name")
print(df.head(20))


    Year  Week    SMN     SMT    VCI    TCI    VHI  area       name
0   2020  51.0  0.082  264.01  69.27  28.14  48.70    24  Вінницька
1   2008  24.0  0.489  295.98  84.96  52.95  68.95    24  Вінницька
2   2008  23.0  0.487  295.36  86.19  55.57  70.88    24  Вінницька
3   2008  22.0  0.481  294.57  88.21  58.62  73.41    24  Вінницька
4   2008  21.0  0.469  293.16  90.28  66.47  78.37    24  Вінницька
5   2008  20.0  0.454  291.96  92.64  70.25  81.44    24  Вінницька
6   2008  19.0  0.436  291.40  95.80  66.20  81.00    24  Вінницька
7   2008  18.0  0.406  290.42  96.43  63.15  79.79    24  Вінницька
8   2008  17.0  0.373  289.64  96.64  54.37  75.51    24  Вінницька
9   2008  16.0  0.336  288.59  96.32  47.04  71.68    24  Вінницька
10  2008  15.0  0.298  287.67  95.79  35.11  65.45    24  Вінницька
11  2008  14.0  0.265  286.50  95.84  25.64  60.74    24  Вінницька
12  2008  13.0  0.236  284.91  95.75  22.84  59.30    24  Вінницька
13  2008  25.0  0.489  296.53  83.41  49.68  66.

In [98]:
display(df)

,Year,Week,SMN,SMT,VCI,TCI,VHI,area,name
0,2020,51.0,0.082,264.01,69.27,28.14,48.70,24,Вінницька
1,2008,24.0,0.489,295.98,84.96,52.95,68.95,24,Вінницька
2,2008,23.0,0.487,295.36,86.19,55.57,70.88,24,Вінницька
3,2008,22.0,0.481,294.57,88.21,58.62,73.41,24,Вінницька
4,2008,21.0,0.469,293.16,90.28,66.47,78.37,24,Вінницька
...,...,...,...,...,...,...,...,...,...
53401,1995,9.0,0.119,281.60,45.14,20.05,32.59,4,Республіка Крим
53402,1995,8.0,0.108,279.35,43.97,21.48,32.72,4,Республіка Крим
53403,1995,7.0,0.095,276.12,39.62,27.95,33.78,4,Республіка Крим
53404,1995,5.0,0.071,266.83,28.87,51.25,40.06,4,Республіка Крим


# Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу)

## 1. Ряд VHI для області за вказаний рік
Отримати часовий ряд **VHI** (Value of the Vegetation Health Index) для однієї конкретної області та одного року.  
Наприклад:
- **Вхідні дані**: назва (або індекс) області та рік.
- **Вихід**: набір (DataFrame) значень VHI за кожен тиждень/місяць протягом вказаного року, а також статистичні показники (середнє, максимум, мінімум).

In [104]:

def vhi_series_and_extremes(df, oblast, year):
 
    subset = df[(df["name"] == oblast) & (df["Year"] == year)]
    
    vhi_values = subset["VHI"]
    
    vhi_min = vhi_values.min()
    vhi_max = vhi_values.max()
    
    print(f"Ряд VHI для області {oblast} за {year} рік:")
    print(vhi_values)
    print(f"Мінімальне VHI: {vhi_min}")
    print(f"Максимальне VHI: {vhi_max}")
    
    return subset


vhi_series_and_extremes(df, "Чернігівська", 2000)


Ряд VHI для області Чернігівська за 2000 рік:
49081    17.80
49082    15.93
49083    15.17
49084    15.80
49085    18.55
49086    21.66
49087    28.51
49088    33.17
49089    37.49
49090    43.62
49091    49.92
49092    55.57
49093    58.22
49094    58.53
49095    59.89
49096    61.96
49097    64.91
49098    67.16
49099    67.09
49100    65.70
49101    62.84
49102    60.02
49103    58.76
49104    58.43
49105    58.40
49106    57.60
49107    57.44
49108    20.09
49109    59.34
49110    20.37
49139    40.89
49141    62.51
49142    63.20
49171    63.82
49174    61.45
49175    56.97
49176    48.74
49177    42.09
49178    39.81
49179    38.72
49180    38.80
49181    38.99
49182    38.87
49183    37.66
49184    36.75
49185    36.45
49186    35.66
49187    32.73
49188    29.62
49189    26.47
49190    24.58
49191    24.84
Name: VHI, dtype: float64
Мінімальне VHI: 15.17
Максимальне VHI: 67.16


,Year,Week,SMN,SMT,VCI,TCI,VHI,area,name
49081,2000,49.0,0.064,271.26,24.75,10.85,17.80,2,Чернігівська
49082,2000,48.0,0.063,272.05,21.03,10.84,15.93,2,Чернігівська
49083,2000,47.0,0.070,273.03,19.60,10.74,15.17,2,Чернігівська
49084,2000,46.0,0.083,274.60,19.78,11.82,15.80,2,Чернігівська
49085,2000,45.0,0.100,276.09,20.21,16.88,18.55,2,Чернігівська
49086,2000,44.0,0.125,277.87,22.38,20.94,21.66,2,Чернігівська
49087,2000,43.0,0.163,279.88,29.93,27.09,28.51,2,Чернігівська
49088,2000,42.0,0.198,281.80,35.15,31.20,33.17,2,Чернігівська
49089,2000,41.0,0.231,283.58,40.48,34.51,37.49,2,Чернігівська
49090,2000,40.0,0.256,284.84,41.64,45.60,43.62,2,Чернігівська


## 2. Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани
Здійснити пошук:
- **Мінімальних та максимальних значень VHI** у вибраних областях та роках.
- **Середнього** та **медіани** VHI за кожну з обраних областей у вказаних роках.  
Наприклад:
- **Вхідні дані**: список областей, список років.
- **Вихід**: таблиця зі стовпцями \[область, рік, min, max, mean, median\].

In [127]:


def max_vhi(df, name, year):
    
    return df[(df["name"] == name) & (df["Year"] == year)]["VHI"].max()

def min_vhi(df, name, year):
   
    return df[(df["name"] == name) & (df["Year"] == year)]["VHI"].min()

def mean_vhi(df, name, year):
  
    return df[(df["name"] == name) & (df["Year"] == year)]["VHI"].mean()

def median_vhi(df, name, year):
   
    return df[(df["name"] == name) & (df["Year"] == year)]["VHI"].median()



v_min_kyiv_2020 = min_vhi(df, "Чернігівська", 1985)
print("Мінімальний VHI для Києва у 2020 році:", v_min_kyiv_2020)
v_max_kyiv_2020 = max_vhi(df, "Чернігівська", 2020)
print("Максимальний VHI для Києва у 2020 році:", v_max_kyiv_2020)
v_median_kyiv_2020 = median_vhi(df, "Чернігівська", 2020)
print("Медіана VHI для Києва у 2020 році:", v_median_kyiv_2020)
v_mean_kyiv_2020 = mean_vhi(df, "Київ", 2019)
print("Медіана VHI для Києва у 2020 році:", v_mean_kyiv_2020)

Мінімальний VHI для Києва у 2020 році: 27.43
Максимальний VHI для Києва у 2020 році: 66.55
Медіана VHI для Києва у 2020 році: 40.625
Медіана VHI для Києва у 2020 році: 42.416538461538465


## 3. Ряд VHI за вказаний діапазон років для вказаних областей
Формування об’єднаного ряду VHI для кількох областей у вказаному **діапазоні років** (наприклад, 2000–2010).  
- **Вхідні дані**: список областей, початковий рік, кінцевий рік.
- **Вихід**: зведений DataFrame із усіма VHI-значеннями за обраний період і список статистичних метрик (min, max, mean, median).

In [101]:
def vhi_series_for_regions_year_range(df, regions, start_year, end_year):
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
    subset = df[
        (df["name"].isin(regions)) &
        (df["Year"] >= start_year) &
        (df["Year"] <= end_year)
    ].copy()

    print(f"Ряд VHI для областей {regions} за період {start_year} - {end_year}:")
    print(subset[["Year", "Week", "name", "VHI"]])

    return subset

vhi_series_for_regions_year_range(df, ["Київська", "Херсонська"], 2000, 2010)


Ряд VHI для областей ['Київська', 'Херсонська'] за період 2000 - 2010:
       Year  Week        name    VHI
16079  2000   1.0    Київська  25.03
16080  2000   2.0    Київська  26.69
16081  2000   3.0    Київська  28.26
16082  2000   4.0    Київська  30.80
16084  2000   5.0    Київська  33.90
...     ...   ...         ...    ...
43253  2000   4.0  Херсонська  32.95
43254  2000   3.0  Херсонська  33.31
43255  2000   2.0  Херсонська  34.07
43256  2000   1.0  Херсонська  34.96
43391  2001  30.0  Херсонська  27.11

[1104 rows x 4 columns]


,Year,Week,SMN,SMT,VCI,TCI,VHI,area,name
16079,2000,1.0,0.031,260.53,9.35,40.70,25.03,11,Київська
16080,2000,2.0,0.033,259.87,11.02,42.35,26.69,11,Київська
16081,2000,3.0,0.034,260.13,14.09,42.44,28.26,11,Київська
16082,2000,4.0,0.037,260.51,18.05,43.56,30.80,11,Київська
16084,2000,5.0,0.040,260.89,20.68,47.13,33.90,11,Київська
...,...,...,...,...,...,...,...,...,...
43253,2000,4.0,0.048,265.11,24.81,41.09,32.95,9,Херсонська
43254,2000,3.0,0.046,263.81,22.66,43.96,33.31,9,Херсонська
43255,2000,2.0,0.043,262.64,19.61,48.53,34.07,9,Херсонська
43256,2000,1.0,0.042,261.68,16.39,53.54,34.96,9,Херсонська


## 4. Виявлення років з екстремальними посухами для певного відсотка областей
Для **всього набору даних** виявити роки, в яких **екстремальні посухи** (низькі VHI) торкнулися більше вказаного відсотка областей.  
- **Приклад**: якщо 20% областей — це 5 областей із 25 можливих, то потрібно повернути всі роки, коли не менше 5 областей мали екстремально низький VHI.  
- **Вихідні дані**:  
  - **Список років**, у яких виконалася умова.  
  - **Назви областей**, що потрапили під екстремальну посуху.  
  - **Значення VHI** для цих областей.  

In [102]:

def find_extreme_drought_years(df: pd.DataFrame, vhi_threshold: float, percentage: float) -> pd.DataFrame:
  

    region_year_min = df.groupby(["Year", "name"])["VHI"].min().reset_index()
    region_year_min["extreme"] = region_year_min["VHI"] < vhi_threshold

    total_regions = df["name"].nunique()
    required_count = math.ceil(total_regions * percentage)

    results = []
    for year, group in region_year_min.groupby("Year"):
        extreme_regions = group[group["extreme"]]
        if len(extreme_regions) >= required_count:
            results.append({
                "Year": year,
                "Affected_regions_count": len(extreme_regions),
                "Regions": list(extreme_regions["name"]),
                "VHI_values": list(extreme_regions["VHI"])
            })

    return pd.DataFrame(results)


vhi_threshold = 15         
percentage_threshold = 0.20  
df_result = find_extreme_drought_years(df, vhi_threshold, percentage_threshold)
display(df_result)


C:\Users\User\AppData\Local\Temp\ipykernel_15648\1730504941.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  region_year_min = df.groupby(["Year", "name"])["VHI"].min().reset_index()


,Year,Affected_regions_count,Regions,VHI_values
0,2000,6,"[Вінницька, Київська, Київ, Севастополь, Харкі...","[11.25, 10.6, 6.49, 8.14, 9.36, 10.68]"
